In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict
from sklearn import ensemble
import random
import datetime

random.seed(2017)

In [18]:
start_time = datetime.datetime.now()

usecols = ['ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

df_train = pd.read_csv('../input/train_ver2.csv', usecols=usecols)
sample = pd.read_csv('../input/sample_submission.csv')

df_train = df_train.drop_duplicates(['ncodpers'], keep='last')
df_train.fillna(0, inplace=True)
df_train=df_train.sample(frac=0.001)


In [17]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = df_train['ncodpers'].values
for c in df_train.columns:
    if c != 'ncodpers':
        print(c)
        y_train = df_train[c]
        x_train = df_train.drop([c, 'ncodpers'], 1)

        clf = ensemble.RandomForestClassifier(n_estimators=10, n_jobs=-1,max_depth=3, min_samples_split=10, verbose=1)

        clf.fit(x_train, y_train)
        p_train1 = clf.predict_proba(x_train)[:,1]


        for id, p in zip(ids, p_train1):
            id_preds[id].append(p)

        print(roc_auc_score(y_train, p_train1))

ind_ahor_fin_ult1


/tmp/ipykernel_191490/2173343078.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_train = df_train.drop([c, 'ncodpers'], 1)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished


IndexError: index 1 is out of bounds for axis 1 with size 1

In [16]:
ids = df_train['ncodpers'].values
id_preds[1049700]

[4.895188824503849e-05,
 0.0,
 0.6017605053997744,
 0.00011758372408615737,
 0.02227141047698755,
 0.025410427923659284,
 0.009217172746508302,
 0.086135584244221,
 0.018819712913914304,
 4.208613015726851e-05,
 9.843748769937972e-05,
 0.011896029012087177,
 0.03503976747874705,
 0.003528843934282266,
 0.0010793341813789005,
 0.0028976633800283777,
 0.0035725244816024395,
 0.020257079593096156,
 0.005248279119474489,
 0.004907969518076183,
 0.0013934295832246824,
 0.00045061092900180766,
 0.0019926865022331436,
 0.016939969280392684]

In [3]:

already_active = {}
for row in df_train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(df_train.columns[1:], row) if c[1] > 0]
    already_active[id] = active

train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    preds = [i[0] for i in sorted([i for i in zip(df_train.columns[1:], p) if i[0] not in already_active[id]], key=lambda i:i [1], reverse=True)[:7]]
    train_preds[id] = preds

test_preds = []
for row in sample.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))

sample['added_products'] = test_preds

sample.to_csv('RFC_sub.csv', index=False)
print(datetime.datetime.now()-start_time)


ind_ahor_fin_ult1


/tmp/ipykernel_191490/2209401020.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_train = df_train.drop([c, 'ncodpers'], 1)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.9s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  60 out of  60 | elapsed:    0.4s finished


0.8394957617267711
ind_aval_fin_ult1


/tmp/ipykernel_191490/2209401020.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_train = df_train.drop([c, 'ncodpers'], 1)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    1.9s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  60 out of  60 | elapsed:    0.4s finished


0.9980036017097537
ind_cco_fin_ult1


...

/tmp/ipykernel_191490/2209401020.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_train = df_train.drop([c, 'ncodpers'], 1)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.8s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  60 out of  60 | elapsed:    0.4s finished


0.998448048516636
ind_recibo_ult1


/tmp/ipykernel_191490/2209401020.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_train = df_train.drop([c, 'ncodpers'], 1)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.8s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  60 out of  60 | elapsed:    0.5s finished


0.8662788336637236


KeyboardInterrupt: 